In [2]:
import os
import numpy as np
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import tempfile
import datetime

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
from IPython.display import display

In [24]:
model_path = 'tmp/gmmgan/'
session_path = None
images_path = None
#session_20200221_0902_stl10_64/images/'


def get_valid_sessions():
    return [name for name in sorted(os.listdir(model_path))
            if 'session' in name 
            and os.path.exists(os.path.join(model_path, name, 'images'))]

def on_session_change(*args):
    global session_path, images_path
    session_path = os.path.join(model_path, get_valid_sessions()[session_slider_w.value])
    images_path = os.path.join(session_path, 'images')
    session_text_w.value = session_path
    on_update_clicked(None)
    
def on_model_update(*args):
    global model_path
    model_path = os.path.join('tmp', model_text_w.value)
    session_slider_w.max = max=len(get_valid_sessions())-1
    session_slider_w.value = 0
    on_session_change(None)

    
def on_update_clicked(_):
    # "linking function with output"
    filenames = [fname for fname in sorted(os.listdir(images_path)) if '.pdf' in fname]
    slider_w.max = max(1, len(filenames) - 1)
    output_w.clear_output()
    if filenames:
        last_file = ':'.join(filenames[-1].split('.')[0].split('_')[1:])
        output_w.append_stdout(
            'max images: {} | last: {}'
            ''.format(len(filenames), last_file))
    

output_w = widgets.Output(layout={'border': '1px solid black'})
                     
model_text_w = widgets.Dropdown(options=['gmmgan', 'wacgan'], value='gmmgan')
session_text_w = widgets.Text(value=os.path.join(model_path, get_valid_sessions()[-1]))
session_slider_w = widgets.IntSlider(max=len(get_valid_sessions())-1)
update_w = widgets.Button(description='Update')
slider_w = widgets.IntSlider()

model_text_w.observe(on_model_update, 'value')
session_slider_w.observe(on_session_change, 'value')
on_update_clicked(None)
update_w.on_click(on_update_clicked)
# displaying button and its output together


def display_img(img_idx):    
    fig, ax = plt.subplots(1,1, figsize=(20,100))
    filenames = [fname for fname in sorted(os.listdir(images_path)) if '.pdf' in fname]
    assert img_idx < len(filenames), 'index out of range'
    images = convert_from_path(os.path.join(images_path, filenames[img_idx]))[0]
    ax.imshow(np.asarray(images))
    #fig.show()

#display_w = interactive(display_img, {'manual': True}, ax=fixed(ax), fig=fixed(fig), img_idx=slider_w)
display_w = interactive(display_img, {'manual': True}, img_idx=slider_w)
widgets.VBox([model_text_w, session_text_w, session_slider_w, output_w, update_w, display_w])